In [2]:
import contextlib
import os
from time import sleep

import qibo
from qibo import gates
from qibo.models.circuit import Circuit

from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration

## Connect to the Qibo Service API

In [3]:
os.environ["QIBOCONNECTION_ENVIRONMENT"] = "development"
configuration = ConnectionConfiguration(username="qili-admin-test", api_key="45843bfc-c404-42ad-abc5-070dk7869420")
qibo_api = API(configuration=configuration)

TypeError: AccessTokenResponse.__init__() missing 1 required positional argument: 'refreshToken'

## Load an already existing connection to the Qilimanjaro Service API

In [ ]:
qibo_api = API()

## Check a connection is alive

In [ ]:
qibo_api.ping()

## List all registered devices

In [ ]:
devices = qibo_api.list_devices()
devices

## Selecting a specific device

In [ ]:
qibo_api.select_device_id(device_id=9)

## Creating an experiment to be executed on a remote device

In [ ]:
qibo.set_backend("numpy")

In [ ]:
circuit = Circuit(1)
circuit.add(gates.H(0))
circuit.add(gates.M(0))

## Remote execution and getting a job identifier

In [ ]:
job_id = qibo_api.execute(circuit=circuit)

In [ ]:
job_id

## User retrieves the Qibo result, but it is still pending

In [ ]:
result = qibo_api.get_result(job_id=job_id)

## When job is executed remotely, a user can retrieve the actual result

In [ ]:
sleep(1)
result = qibo_api.get_result(job_id=job_id)

In [ ]:
result.state()

In [ ]:
result.frequencies(binary=True, registers=True)

In [ ]:
circuit.draw()

In [ ]:
qibo_api.release_device(device_id=9)

In [ ]:
devices._devices[0].id

## Block and release a remote device to operate on it

In [ ]:
qibo_api.block_device_id(device_id=1)

# Do stuff with the device, knowing no one else will access it

qibo_api.release_device(device_id=1)

## Trying to block a blocked device will raise an exception

In [ ]:
with contextlib.suppress(Exception):
    qibo_api.block_device_id(device_id=1)
    qibo_api.block_device_id(device_id=1)

qibo_api.release_device(device_id=1)